## NN model for X-ray Body Part Classifier

In [ ]:
import os
from google.colab import drive
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, BatchNormalization, AveragePooling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### Download CSV files with features (X) and target (y)

In [ ]:
X = pd.read_csv('/content/drive/MyDrive/train_X.csv', header=None)

In [ ]:
y = pd.read_csv('/content/drive/MyDrive/train_y.csv')

### Show classes and number of instances in them

In [ ]:
len(y['0'].unique())

41

In [ ]:
y.groupby(['0'])['0'].count()

0
0      80
1      41
2      77
3     724
4       9
5      23
6      70
7      15
8      12
9      31
10     19
11    102
12     19
13     39
14    120
15     67
16     40
17     23
18     10
19      7
20     15
21     63
22      2
23     45
24     11
25      7
26     42
27      1
28      3
29      1
30      1
31      5
32      1
33      2
34      4
35      1
36      2
37      1
38      1
39      1
40      1
Name: 0, dtype: int64

##### The classes are imbalanced, one way to avoid overfitting is to use the RandomOverSampler.

In [ ]:
y = list(y['0'])                                              # make y (target)

In [ ]:
X = pd.DataFrame(X).to_numpy()                                # make X (features)

In [ ]:
ros = RandomOverSampler( sampling_strategy = 'minority')
X_resampled, y_resampled = ros.fit_resample (X, y)            # create similar ones in minority classes

In [ ]:
print('Rows number before RandomOverSampler:', X.shape[0])
print('Rows number after RandomOverSampler:', X_resampled.shape[0])

Rows number before RandomOverSampler: 1738
Rows number after RandomOverSampler: 2461


In [ ]:
X_resampled = np.reshape(X_resampled, (2461, 84, 84))         # return values to image matrix

### Split data for training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.1, random_state = 45) 

### Make categories for target

In [ ]:
y_train_cat = keras.utils.to_categorical(y_train, 41)                          
y_test_cat = keras.utils.to_categorical(y_test, 41)

### Prepare arrays with features

In [ ]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

### Make model

In [ ]:
epo = 52                                               # number of epochs
batch = 32
l2_lambda = 0.0002                                     # L2-regularisation
myOpt = keras.optimizers.Adam(learning_rate = 0.0003)

### Create a convolutional neural network

In [ ]:
model = keras.Sequential([
    Conv2D(32, (3,3),
           padding = 'same',
           activation = 'relu',
           kernel_regularizer = l2(l2_lambda),
           input_shape = (84, 84, 1)),
    AveragePooling2D(pool_size = (4, 4)),
    
    Conv2D(64, (3,3), padding = 'same',
           activation='relu'),
    BatchNormalization (),
    AveragePooling2D(pool_size = (4, 4)),
    Flatten(),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(41,  activation = 'softmax')
])

model.compile(optimizer = myOpt,
             loss = 'categorical_crossentropy',
             metrics = 'accuracy')


his = model.fit(X_train, y_train_cat,
                batch_size = batch,
                epochs = epo,
                validation_split = 0.1,)

model.evaluate(X_test, y_test_cat)

Epoch 1/52
63/63 [==============================] - 1s 10ms/step - loss: 1.7892 - accuracy: 0.5628 - val_loss: 5.6476 - val_accuracy: 0.0586
Epoch 2/52
63/63 [==============================] - 0s 7ms/step - loss: 1.2257 - accuracy: 0.6827 - val_loss: 2.3453 - val_accuracy: 0.4730
Epoch 3/52
63/63 [==============================] - 0s 7ms/step - loss: 1.0755 - accuracy: 0.7209 - val_loss: 1.6682 - val_accuracy: 0.6532
Epoch 4/52
63/63 [==============================] - 0s 7ms/step - loss: 0.9794 - accuracy: 0.7405 - val_loss: 1.5365 - val_accuracy: 0.4189
Epoch 5/52
63/63 [==============================] - 0s 7ms/step - loss: 0.8919 - accuracy: 0.7631 - val_loss: 1.1728 - val_accuracy: 0.7117
Epoch 6/52
63/63 [==============================] - 0s 7ms/step - loss: 0.8366 - accuracy: 0.7736 - val_loss: 1.0933 - val_accuracy: 0.7432
Epoch 7/52
63/63 [==============================] - 0s 7ms/step - loss: 0.8112 - accuracy: 0.7856 - val_loss: 1.1442 - val_accuracy: 0.7162
Epoch 8/52
63/63 [=

[0.8171331882476807, 0.8663967847824097]

In [ ]:
predictions = model.predict(X_test)
print('Real values: \n', np.array(y_test))
print('Predict values: \n', y_pred)
print('Accuracy score: ', accuracy_score(y_test, y_pred))

Real values: 
 [27 11  3  3  1  3 26 27 27  3 27  0 15  3  3  3  0  3 27  3  3 27 14 27
 16  1  3  3 27  3  2 16  3  3  3  0 27 23  3  1 11 11 23  6 27  0 11 27
 23  3  3 27 27  0 27 27 27  0  3  3  3  2 14 21 27  3  0  3 27 23 11  3
  3  3 27 17  3  3 27  3  3 27  9  3  3 16 14 27  3  3  3  3 27 19  3 21
  3 13  3  0 27  3 26 11  3 23  2 16  6  3 11  3 10  1 27 11 27 20  3 27
  1 27  3  3 11 32  2 27  3  3  3  3  2 27  3 36  2  0 17 23 27 27 27 26
 27  3  3 27 27 27  6 11  3 27 11  3 27  3 14 27  3  3 27 11 27 23 27 13
  3  3 21 19 21  6  3  3 27 11 11 27 27 11 13 27 17  3 27  3  3 11  7 27
 27  5  3 14 26  3  3 27  6 27 27  3 26 27 27  3 14  3  2  0 14 27  1 11
 26  3  3  3 21 27  3 34 27 27 27  3 27  3  3 27 27 21  3  3 27  3 26  3
 21  3 27 35  3 11 11]
Predict values: 
 [27 11  3  3  6  3 26 27 27  3 27 11 15  3  3  3  0  3 27  3  3 27 14 27
 16  1  3  3 27  3 10 16  3  3  3  0 27 14  3  1 11 11 23  7 27  3  5 27
 23  3  3 27 27  0 27 27 27  0  3  3  3  3 14  1 27  3  0  3 27 23 1

##### The model predicts X-ray Body Part with an accuracy of 86.6%.